In [1]:
# import ragas.metrics as m
import os
from dotenv import load_dotenv
import pandas as pd
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

load_dotenv()
link = os.getenv('dsa_2214')
token = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = token

chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

In [2]:
data = pd.read_csv('data/sorting.csv')
data.columns = ['concept', 'outcome']
data

,concept,outcome
0,Data Structures->Basics->Algorithm Analysis,Apply time complexity analysis guideline to an...
1,Data Structures->Basics->Algorithm Analysis->O...,Demonstrate an understanding of big O notation...
2,Algorithms->Non-recursive Algorthims,Demonstrate an understanding of non-recursive ...
3,Algorithms->Non-recursive Algorithms->Search,Apply the Comparable interface for object comp...
4,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of linear search;...
5,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of binary search;...
6,Algorithms->Non-recursive Algorithms->Sort,Demonstrate an understanding of sorting;
7,Algorithms->Non-recursive Algorithms->Sort->In...,Demonstrate an understanding of insertion sort...
8,Algorithms->Non-recursive Algorithms->Sort->Se...,Demonstrate an understanding of selection sort...
9,Algorithms->Non-recursive Algorithms->Sort->Bu...,Demonstrate an understanding of bubble sort;An...


In [3]:
concept_data = data['concept'].tolist()
actual_concepts = []
for string in concept_data:
    words = string.split('->')
    for word in words:
        if word not in actual_concepts:
            actual_concepts.append(word)


outcome_data = data['outcome'].tolist()
actual_outcomes = []
for string in outcome_data:
    words = string.split(';')
    for s in words:
        if s not in actual_outcomes:
            actual_outcomes.append(s)

actual_concepts = [' '.join(actual_concepts)] * 4
actual_outcomes = [' '.join(actual_outcomes)] * 4

In [4]:
from src.extractor import relationExtractor
extractor = relationExtractor(link, token, chapters[6:10], os.getenv('connection_string'), 3000, 100, 'DocumentEmbeddings', '2214_embeddings')

/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


<h4>Testing deepeval...</h4>

In [5]:
retrieved, concepts = extractor.identify_concepts(5)

In [6]:
from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric, FaithfulnessMetric
metrics = [AnswerRelevancyMetric(model = 'gpt-4o-mini'), ContextualPrecisionMetric(model = 'gpt-4o-mini'), ContextualRecallMetric(model = 'gpt-4o-mini'), FaithfulnessMetric(model = 'gpt-4o-mini')]
samples = extractor.evaluate('concepts', 5, concepts, actual_concepts, retrieved, metrics)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 4 test case(s) in parallel: |██████████|100% (4/4) [Time Taken: 00:17,  4.49s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The score is 1.00 because there were no irrelevant statements present in the output, effectively addressing the input query on learning concepts for File Processing and External Sorting., error: None)
  - ✅ Contextual Precision (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The score is 1.00 because all relevant nodes are ranked higher than the irrelevant nodes. The first node highlights 'a detailed explanation of Mergesort', while the second node discusses 'the principles of external sorting', both key to the expected output. The irrelevant nodes rank lower because they 'do not mention any specific algorithms' or 'are primarily focused on sorting algorithms and disk I/O', making them less relevant to the learning concepts needed., error: None)
  - ❌ Contextual Recall (score: 0.125, threshold: 0.5, strict: False, evaluation mod

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

test_results=[TestResult(name='test_case_1', success=False, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because there were no irrelevant statements present in the output, effectively addressing the input query on learning concepts for File Processing and External Sorting.', strict_mode=False, evaluation_model='gpt-4o-mini', error=None, evaluation_cost=0.0011377499999999999, verbose_logs='Statements:\n[\n    "Here are the five most important learning concepts for the chapter on File Processing and External Sorting.",\n    "Understanding External Sorting.",\n    "External sorting is crucial for handling large datasets that do not fit into main memory.",\n    "It involves reading data from disk, processing it in blocks, and writing it back efficiently.",\n    "Key algorithms, such as Mergesort, are adapted for external sorting to minimize disk I/O operations.",\n    "Disk I/O operations are significantly slower than m

In [7]:
from src.utils import normalize_text

normalized_concepts = [[normalize_text(' '.join(t))] for t in concepts]
normalized_truths = [normalize_text(t) for t in actual_concepts]

normalized_retrieved = {}
for k in retrieved.keys():
    normalized_retrieved[k] = normalize_text(retrieved[k])

# for i in range(5):
normalized_samples = extractor.evaluate('concepts', 5, normalized_concepts, normalized_truths, data = normalized_retrieved, metrics = metrics)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 4 test case(s) in parallel: |██████████|100% (4/4) [Time Taken: 00:17,  4.48s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.9375, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The score is 0.94 because while the output contains valuable information about mergesort, it includes a general statement that does not specifically address the chapter's key concepts, slightly detracting from its relevancy., error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The score is 0.00 because the relevant nodes are not present in the retrieval context. The first node primarily discusses mergesort and internal sorting, lacking references to key concepts like 'data structure', 'basic algorithm analysis', or 'nonrecursive algorithms'. These omissions illustrate that the irrelevant nodes do not meet the criteria for ranking, resulting in a poor contextual precision score., error: None)
  - ✅ Contextual Recall (score: 0.9230769230769231, threshold: 0.5, strict: False, evaluation model:

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

test_results=[TestResult(name='test_case_1', success=False, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=0.9375, reason="The score is 0.94 because while the output contains valuable information about mergesort, it includes a general statement that does not specifically address the chapter's key concepts, slightly detracting from its relevancy.", strict_mode=False, evaluation_model='gpt-4o-mini', error=None, evaluation_cost=0.00100815, verbose_logs='Statements:\n[\n    "five important learning concept chapter file processing external sorting",\n    "understanding external sorting",\n    "external sorting crucial handling large datasets fit main memory",\n    "involves reading data disk, processing block, writing back efficiently",\n    "key algorithm, mergesort, adapted external sorting minimize disk i/o operation, significantly slower memory access",\n    "mergesort algorithm",\n    "mergesort fundamental sorting algorithm work recursively dividi